# Prototype NN model

# Import 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/interim_data/clean_ds.csv') # for semplicity i use the clean data (not the normalized one)

In [3]:
df.head()

,aggregated1,aggregated2,granular1,granular2,granular3,granular4,granular5,id
0,8.00,9.00,8.00,8.00,9.00,8.00,8.00,1
1,6.67,8.33,6.00,7.67,7.00,7.00,6.67,2
2,5.33,4.33,6.33,4.00,6.33,4.33,6.67,3
3,7.67,8.33,7.67,8.33,6.67,7.67,8.67,4
4,7.00,8.33,7.67,8.00,9.00,7.67,8.33,5


# Model

In [15]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

Define target and features

In [10]:
target_cols = ['aggregated1', 'aggregated2']
feature_cols = ['granular1', 'granular2', 'granular3', 'granular4', 'granular5']

X = df[feature_cols].values
y = df[target_cols].values

Define train and test set

In [13]:
seed = 0
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=seed)

Define the model

In [17]:
class gilizaNN(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.net = nn.Sequential(
            nn.Linear(5, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )
    
    def forward(self, x):
        return self.net(x)

model = gilizaNN()

Train model (see tier)